bayesian-optimization 설치 필요

In [1]:
# !pip install -q bayesian-optimization

### Load Data
- PSM
    * eBay의 여러 애플리케이션 서버 노드에서 내부적으로 21주 동안 수집
    * 25개의 센서로 수집한 다변량 시계열 데이터
    * 훈련: 13주, 테스트 8주
- 원본의 PSM 데이터셋에서 결측치 제거, numpy array 형태로 저장

In [3]:
import numpy as np

X_train = np.load('PSM/X_train.npy')
X_test = np.load('PSM/X_test.npy')
labels = np.load('PSM/labels.npy')

print(X_train.shape)
print(X_test.shape)
print(labels.shape)

(129784, 25)
(87841, 25)
(87841,)


In [4]:
import torch

device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cpu'

### 모델 정의
- 하이퍼파라미터 num_kernels를 최소화하는 것이 목적
- 초기 num_kernels는 6으로 설정됨에 따라 6개의 Convolution Layer을 사용하고, 각각 크기가 1x1, 3x3, 5x5, 7x7, 9x9, 11x11, 13x13인 kernel 사용

In [9]:
from timesnet.timesnet import TimesNet

clf = TimesNet(seq_len=100,
                 stride=1,
                 lr=0.0001,
                 epochs=10,
                 batch_size=128,
                 epoch_steps=20,
                 prt_steps=1,
                 device=device,
                 pred_len=0,
                 e_layers=2,
                 d_model=64,
                 d_ff=64,
                 dropout=0.1,
                 top_k=3,
                 num_kernels=6, # target
                 verbose=2,
                 random_state=42)

### 베이지안 최적화
- 모델을 학습하는 런타임 중에 하이퍼파라미터를 튜닝
- 목적 함수: θ, n = argmax(AUC(y, ϕ(θ)) - αn)
- $\theta, n = argmax(AUC(y, \phi(\theta)) - \alpha n)$
- <img src="./objective_function.png" width="400">
- 어떤 형식이 예쁠까요?

In [ ]:
from bayes_opt import BayesianOptimization
from metrics import auc_score

def objective_function(num_kernels):
    clf.num_kernels = int(num_kernels)
    clf.fit(X_train)
    scores = clf.decision_function(X_test)
    roc_auc = auc_score(labels, scores)
    return roc_auc-0.01*int(num_kernels)

pbounds = {'num_kernels':(1, 6)}
optimizer = BayesianOptimization(
    f=objective_function,
    pbounds=pbounds,
    verbose=2,
    random_state=42
)

optimizer.maximize(init_points=3, n_iter=3)

print("최적의 커널 개수:", optimizer.max['params'])
print("최적의 목적 함수 값:", optimizer.max['target'])

In [ ]:
best_num_kernels = int(optimizer.max['params']['num_kernels'])

best_clf = TimesNet(seq_len=100,
                 stride=1,
                 lr=0.0001,
                 epochs=10,
                 batch_size=128,
                 epoch_steps=20,
                 prt_steps=1,
                 device=device,
                 pred_len=0,
                 e_layers=2,
                 d_model=64,
                 d_ff=64,
                 dropout=0.1,
                 top_k=3,
                 num_kernels=best_num_kernels, # target
                 verbose=2,
                 random_state=42)

# 최적의 하이퍼파라미터로 모델 학습
best_clf.fit(X_train)

### 테스트

In [ ]:
def get_model_size(model):
    size_model = 0
    for param in best_clf.net.parameters():
        if param.data.is_floating_point():
            size_model += param.numel() * torch.finfo(param.data.dtype).bits
        else:
            size_model += param.numel() * torch.iinfo(param.data.dtype).bits
    print(f"model size: {size_model} / bit | {size_model / 8e6:.2f} / MB")
    
    return

In [ ]:
from metrics import ts_metrics, point_adjustment

scores = best_clf.decision_function(X_test)

eval_metrics = ts_metrics(labels, scores)
adj_eval_metrics = ts_metrics(labels, point_adjustment(labels, scores))

print(f"kernel : {best_clf.num_kernels}")
print(eval_metrics)
print(adj_eval_metrics) # point adjustment 실행 후 AUC score, Precision-Recall score, f1, precision, recall
get_model_size(best_clf.net)